In [1]:

dft93 = 'Data\\traffic_count\dft_rawcount_local_authority_id_93.csv'
dft96 = 'Data\\traffic_count\dft_rawcount_local_authority_id_96.csv'
dft109 = 'Data\\traffic_count\dft_rawcount_local_authority_id_109.csv'
dft110 = 'Data\\traffic_count\dft_rawcount_local_authority_id_110.csv'
dft145 = 'Data\\traffic_count\dft_rawcount_local_authority_id_145.csv'
dft174 = 'Data\\traffic_count\dft_rawcount_local_authority_id_174.csv'


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Chailee\AppData\Local\Temp\ipykernel_27736\2460736172.py:1: SyntaxWarning: invalid escape sequence '\d'
  dft93 = 'Data\\traffic_count\dft_rawcount_local_authority_id_93.csv'
C:\Users\Chailee\AppData\Local\Temp\ipykernel_27736\2460736172.py:2: SyntaxWarning: invalid escape sequence '\d'
  dft96 = 'Data\\traffic_count\dft_rawcount_local_authority_id_96.csv'
C:\Users\Chailee\AppData\Local\T

In [2]:
import pandas as pd

# 定义文件路径列表
file_paths = [
    'Data\\traffic_count\\dft_rawcount_local_authority_id_93.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_96.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_109.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_110.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_145.csv',
    'Data\\traffic_count\\dft_rawcount_local_authority_id_174.csv'
]

# 指定需要过滤的年份
years_to_filter = [2019, 2021, 2023, 2024]

for file_path in file_paths:
    # 读取CSV文件
    df = pd.read_csv(file_path)
    
    # 将count_date列转换为datetime类型
    df['count_date'] = pd.to_datetime(df['count_date'])
    
    # 过滤出指定年份的行
    filtered_df = df[df['count_date'].dt.year.isin(years_to_filter)]
    
    # 构建新的文件名
    new_file_path = file_path.replace('.csv', '_filtered.csv')
    
    # 保存过滤后的数据到新文件
    filtered_df.to_csv(new_file_path, index=False)

    print(f"Filtered data saved to {new_file_path}")

Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_93_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_96_filtered.csv


C:\Users\Chailee\AppData\Local\Temp\ipykernel_27736\327191506.py:18: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_109_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_110_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_145_filtered.csv
Filtered data saved to Data\traffic_count\dft_rawcount_local_authority_id_174_filtered.csv
